In [1]:
import pandas as pd
import numpy as np
import pickle

from utilities.config import *
from utilities.utils import *
from gensim.models import Word2Vec, KeyedVectors

import pymysql

In [2]:
# Unpickling
with open("./model/job_skill_corpus", "rb") as fp:
    job_skill_list = pickle.load(fp)

In [20]:
cnx = pymysql.connect(
    host=CONFIG_NAT['host'],
    user=CONFIG_NAT['user'],
    password=CONFIG_NAT['password'],
    db=CONFIG_NAT['database'],
    charset = 'utf8',
    cursorclass=pymysql.cursors.DictCursor)
cursor = cnx.cursor()

In [13]:
model = Word2Vec(sentences=job_skill_list, size=200, window=30, min_count=1, workers=4)
model.save("./model/nat_job_skill_wordvec.model")

In [14]:
print(f"The model has {len(model.wv.vocab)} unique skill vectors.")

2414

In [23]:
# model.wv.most_similar(positive=['big data analytics','python'], negative=['plotly'], topn=5)
model.wv.most_similar(positive=['adobe xd'], topn=5)
# model.wv.most_similar(positive=['python'], topn=5)

[('invision', 0.8040086030960083),
 ('figma', 0.7910892367362976),
 ('framer (software)', 0.7706668376922607),
 ('axure rp', 0.7687813639640808),
 ('adobe dreamweaver', 0.7440941333770752)]

### Insert into DB

In [16]:
skill_vector_df = pd.DataFrame()
for skill, attrib in model.wv.vocab.items():
    skill_vec = model.wv[skill]
    skill_vector_df = skill_vector_df.append({'skill':skill, 'skill_vectors':skill_vec}, ignore_index=True)

In [17]:
skill_vector_df.head()

,skill,skill_vectors
0,sales and business development,"[0.03194156, 0.12575376, 0.9065852, -1.961084,..."
1,engineering problem solving,"[-1.2087154, -0.9381196, 0.20606227, -0.012821..."
2,applications support and enhancement,"[-0.20772204, -1.2196033, 1.1390424, 2.875674,..."
3,process control,"[-2.5633702, -1.3956965, 0.03706379, -0.345627..."
4,business opportunities development,"[0.502539, 0.900933, 0.24350785, 0.91363716, -..."


In [25]:
insert_skill_vec_statmt = ("""
              INSERT IGNORE INTO `nat_poc`.`skill_vectors`
                (`skill`, `dim_1`, `dim_2`, `dim_3`, `dim_4`, `dim_5`, `dim_6`, `dim_7`, `dim_8`, `dim_9`, `dim_10`, `dim_11`, `dim_12`, `dim_13`, `dim_14`, `dim_15`, `dim_16`, `dim_17`, `dim_18`, `dim_19`, `dim_20`, `dim_21`, `dim_22`, `dim_23`, `dim_24`, `dim_25`, `dim_26`, `dim_27`, `dim_28`, `dim_29`, `dim_30`, `dim_31`, `dim_32`, `dim_33`, `dim_34`, `dim_35`, `dim_36`, `dim_37`, `dim_38`, `dim_39`, `dim_40`, `dim_41`, `dim_42`, `dim_43`, `dim_44`, `dim_45`, `dim_46`, `dim_47`, `dim_48`, `dim_49`, `dim_50`, `dim_51`, `dim_52`, `dim_53`, `dim_54`, `dim_55`, `dim_56`, `dim_57`, `dim_58`, `dim_59`, `dim_60`, `dim_61`, `dim_62`, `dim_63`, `dim_64`, `dim_65`, `dim_66`, `dim_67`, `dim_68`, `dim_69`, `dim_70`, `dim_71`, `dim_72`, `dim_73`, `dim_74`, `dim_75`, `dim_76`, `dim_77`, `dim_78`, `dim_79`, `dim_80`, `dim_81`, `dim_82`, `dim_83`, `dim_84`, `dim_85`, `dim_86`, `dim_87`, `dim_88`, `dim_89`, `dim_90`, `dim_91`, `dim_92`, `dim_93`, `dim_94`, `dim_95`, `dim_96`, `dim_97`, `dim_98`, `dim_99`, `dim_100`, `dim_101`, `dim_102`, `dim_103`, `dim_104`, `dim_105`, `dim_106`, `dim_107`, `dim_108`, `dim_109`, `dim_110`, `dim_111`, `dim_112`, `dim_113`, `dim_114`, `dim_115`, `dim_116`, `dim_117`, `dim_118`, `dim_119`, `dim_120`, `dim_121`, `dim_122`, `dim_123`, `dim_124`, `dim_125`, `dim_126`, `dim_127`, `dim_128`, `dim_129`, `dim_130`, `dim_131`, `dim_132`, `dim_133`, `dim_134`, `dim_135`, `dim_136`, `dim_137`, `dim_138`, `dim_139`, `dim_140`, `dim_141`, `dim_142`, `dim_143`, `dim_144`, `dim_145`, `dim_146`, `dim_147`, `dim_148`, `dim_149`, `dim_150`, `dim_151`, `dim_152`, `dim_153`, `dim_154`, `dim_155`, `dim_156`, `dim_157`, `dim_158`, `dim_159`, `dim_160`, `dim_161`, `dim_162`, `dim_163`, `dim_164`, `dim_165`, `dim_166`, `dim_167`, `dim_168`, `dim_169`, `dim_170`, `dim_171`, `dim_172`, `dim_173`, `dim_174`, `dim_175`, `dim_176`, `dim_177`, `dim_178`, `dim_179`, `dim_180`, `dim_181`, `dim_182`, `dim_183`, `dim_184`, `dim_185`, `dim_186`, `dim_187`, `dim_188`, `dim_189`, `dim_190`, `dim_191`, `dim_192`, `dim_193`, `dim_194`, `dim_195`, `dim_196`, `dim_197`, `dim_198`, `dim_199`, `dim_200`)
                VALUES
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """)

In [24]:
col_names = ['skill'] + ['dim_'+str(i) for i in range(1, 200+1)]
vector_list_of_list = [i for i in skill_vector_df[['skill','skill_vectors']].apply(lambda x: [x[0]] + list(x[1]), axis=1)]

In [30]:
skill_vector_sql = pd.DataFrame(vector_list_of_list, columns=col_names)

In [31]:
cursor.executemany(insert_skill_vec_statmt, zip(*[skill_vector_sql[col].values.tolist() for col in skill_vector_sql]) )

2414

In [33]:
cnx.commit()

In [24]:
def close_db():
    cnx.close()
    cursor.close()

In [27]:
close_db()